In [ ]:
# 回测：Bybit 订单簿数据

[NautilusTrader](https://nautilustrader.io/docs/) 高性能算法交易平台和事件驱动回测器教程。

[在GitHub上查看源码](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/tutorials/backtest_bybit_orderbook.ipynb)。

:::info
我们目前正在完善此教程。
:::


In [ ]:
## 概述

本教程介绍如何设置数据目录和`BacktestNode`来回测订单簿数据上的`OrderBookImbalance`策略。此示例需要Bybit提供的订单簿深度数据。


In [ ]:
## 先决条件

- 已安装 Python 3.11+
- 已安装 [JupyterLab](https://jupyter.org/) 或类似工具 (`pip install -U jupyterlab`)
- 已安装 [NautilusTrader](https://pypi.org/project/nautilus_trader/) 最新版本 (`pip install -U nautilus_trader`)


In [ ]:
## 导入

我们先导入本指南所需的所有模块：


In [ ]:
import os
import shutil
from decimal import Decimal
from pathlib import Path

import pandas as pd

from nautilus_trader.adapters.bybit.loaders import BybitOrderBookDeltaDataLoader
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model import OrderBookDelta
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import OrderBookDeltaDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider


In [ ]:
## 加载数据


In [ ]:
# 数据目录路径，以用户/Downloads为例
DATA_DIR = "~/Downloads"


In [ ]:
data_path = Path(DATA_DIR).expanduser() / "Data" / "Bybit"
raw_files = list(data_path.iterdir())
assert raw_files, f"Unable to find any histdata files in directory {data_path}"
raw_files


In [ ]:
# 我们将使用Bybit提供的订单簿深度500数据，限制为100万行
path_update = data_path / "2024-12-01_XRPUSDT_ob500.data.zip"
nrows = 1_000_000
df_raw = BybitOrderBookDeltaDataLoader.load(path_update, nrows=nrows)
df_raw.head()


In [ ]:
### 使用数据处理器处理增量数据


In [ ]:
XRPUSDT_BYBIT = TestInstrumentProvider.xrpusdt_linear_bybit()
wrangler = OrderBookDeltaDataWrangler(XRPUSDT_BYBIT)

deltas = wrangler.process(df_raw)
deltas.sort(key=lambda x: x.ts_init)  # 确保数据按`ts_init`非递减排序
deltas[:10]


In [ ]:
### 设置数据目录


In [ ]:
CATALOG_PATH = os.getcwd() + "/catalog"

# 如果目录已存在则清除，然后创建新的
if os.path.exists(CATALOG_PATH):
    shutil.rmtree(CATALOG_PATH)
os.mkdir(CATALOG_PATH)

# 创建目录实例
catalog = ParquetDataCatalog(CATALOG_PATH)


In [ ]:
# 将工具和tick数据写入目录
catalog.write_data([XRPUSDT_BYBIT])
catalog.write_data(deltas)


In [ ]:
# 确认工具已写入
catalog.instruments()


In [ ]:
# 探索目录中的可用数据
start = dt_to_unix_nanos(pd.Timestamp("2022-11-01", tz="UTC"))
end =  dt_to_unix_nanos(pd.Timestamp("2022-11-04", tz="UTC"))

deltas = catalog.order_book_deltas(start=start, end=end)
print(len(deltas))
deltas[:10]


In [ ]:
## 配置回测


In [ ]:
instrument = catalog.instruments()[0]
book_type = "L2_MBP"  # 确保数据订单簿类型与交易场所订单簿类型匹配

data_configs = [BacktestDataConfig(
        catalog_path=CATALOG_PATH,
        data_cls=OrderBookDelta,
        instrument_id=instrument.id,
        # start_time=start,  # 运行所有数据
        # end_time=end,  # 运行所有数据
    )
]

venues_configs = [
    BacktestVenueConfig(
        name="BYBIT",
        oms_type="NETTING",
        account_type="CASH",
        base_currency=None,
        starting_balances=["200000 XRP", "100000 USDT"],
        book_type=book_type,  # <-- 交易场所订单簿类型
    )
]

strategies = [
    ImportableStrategyConfig(
        strategy_path="nautilus_trader.examples.strategies.orderbook_imbalance:OrderBookImbalance",
        config_path="nautilus_trader.examples.strategies.orderbook_imbalance:OrderBookImbalanceConfig",
        config={
            "instrument_id": instrument.id,
            "book_type": book_type,
            "max_trade_size": Decimal("1.000"),
            "min_seconds_between_triggers": 1.0,
        },
    ),
]

# NautilusTrader目前超过了Jupyter notebook日志记录的速率限制（stdout输出），
# 这就是为什么`log_level`设置为"ERROR"的原因。如果您降低此级别以查看
# 更多日志记录，那么notebook将在cell执行期间挂起。目前正在
# 调查一个修复方案，该方案涉及提高Jupyter的配置速率限制，
# 或者限制来自Nautilus的日志刷新。
# https://github.com/jupyterlab/jupyterlab/issues/12845
# https://github.com/deshaw/jupyterlab-limit-output
config = BacktestRunConfig(
    engine=BacktestEngineConfig(
        strategies=strategies,
        logging=LoggingConfig(log_level="ERROR"),
    ),
    data=data_configs,
    venues=venues_configs,
)

config


In [ ]:
## 运行回测


In [ ]:
node = BacktestNode(configs=[config])

result = node.run()


In [ ]:
result


In [ ]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue


engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()


In [ ]:
engine.trader.generate_positions_report()


In [ ]:
engine.trader.generate_account_report(Venue("BYBIT"))
